In [ ]:
import tensorflow as tf
from tensorflow.keras import datasets, layers, models
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import load_model

import keras
from keras.models import Sequential, Model
from keras.layers import *
from keras.utils import Sequence
from keras.layers import Conv2D, MaxPooling2D
from keras.utils import Sequence
from keras.callbacks import CSVLogger
from keras.callbacks import EarlyStopping

from qkeras import *
from qkeras.utils import _add_supported_quantized_objects

import matplotlib.pyplot as plt

import os
import json
import random

pi = 3.14159265359

maxval = 1e9
minval = 1e-9

In [ ]:
#from dataprep import *
from dataloaders.OptimizedDataGenerator import OptimizedDataGenerator
from loss import *

In [ ]:
# Enable batch normalization in the model (True, False)
batch_norm_enabled = False
# True = use all of the time slices, False = use a subset of the timeslices
timeslices_all_enable = False
# True = load tfrecords, False = (re-)generated tfrecords
load_from_tfrecords_enabled = False
# True = load model from file, False = train it from scratch
load_model_from_hdf5_enabled = False

In [ ]:
if batch_norm_enabled:
    from models_bnorm import *
else:
    from models import *

# Scaling Lists for Different Pixel Pitches:
* 100x25x100 um:  [150.0, 37.5, 10.0, 1.22]
* 50x25x100 um:   [75.0, 37.5, 10.0, 1.22]
* 50x20x100 um:   [75.0, 30.0, 10.0, 1.22]
* 50x15x100 um:   [75.0, 22.5, 10.0, 1.22]
* 50x12.5x100 um: [75.0, 18.75, 10.0, 1.22]
* 50x10x100 um:   [75.0, 15.0, 10.0, 1.22]

In [ ]:
# You can define a JSON configuration file locally
# {
#    "data_base_dir": "/data/dajiang/smartPixels",
#    "tfrecords_base_dir" : "/data/dajiang/smartPixels",
#    "model_base_dir": "/home/dajiang/smart-pixels-ml/weights"
# }
config_file_path = 'config.json'

# If the file does not exist, the notebook uses default values for those entries
data_base_dir = "/data/dajiang/smartPixels/dataset_2s"
tfrecords_base_dir = "/data/dajiang/smartPixels/tfrecords"
npy_base_dir = "/data/dajiang/smartPixels/npy"
model_base_dir = "/home/dajiang/smart-pixels-ml/weights"

if os.path.exists(config_file_path):
    with open(config_file_path, 'r') as file:
        data = json.load(file)
        data_base_dir = data.get('data_base_dir')
        tfrecords_base_dir = data.get('tfrecords_base_dir')
        npy_base_dir = data.get('npy_base_dir')
        model_base_dir = data.get('model_base_dir')
    print(f"Use config info from file: {data_base_dir}, {tfrecords_base_dir}, {npy_base_dir}, {model_base_dir}")
else:
    print(f"File does not exist. Use default config info: {data_base_dir}, {tfrecords_base_dir}, {npy_base_dir}, {model_base_dir}")

In [ ]:
%%time

batch_size = 1000
val_batch_size = 1000
train_file_size = 50
val_file_size = 10

# See: https://docs.google.com/document/d/1ZoqVyJOOAXhzt2egMWh3OoNJ6lWq5lNR6sjcYON4Vlo/edit?tab=t.0#heading=h.k6tyal7z5t5l
dataset_name = "dataset_2s"
# 50x12.5x100 micron pixel sensor => 13x21 pixel sensor array
sensor_geometry_name = "50x12P5x100"
# Either 20 or 2 timeslices
timeslices_name = "timeslices20" if timeslices_all_enable else "timeslices2"
timeslices_val = -1 if timeslices_all_enable else [0, 19]
#
batch_size_name = f"bs{batch_size}"

# Input: parquets
data_dir = f"{data_base_dir}/dataset_2s_50x12P5_parquets/unflipped/recon3D/"
labels_dir = f"{data_base_dir}/dataset_2s_50x12P5_parquets/unflipped/labels/"

# Output: tfrecords
tfrecords_dir_train = f"{tfrecords_base_dir}/tfrecords_{dataset_name}_{sensor_geometry_name}_{timeslices_name}_{batch_size_name}_train"
tfrecords_dir_val = f"{tfrecords_base_dir}/tfrecords_{dataset_name}_{sensor_geometry_name}_{timeslices_name}_{batch_size_name}_val"

training_generator = OptimizedDataGenerator(
    data_directory_path = data_dir,
    labels_directory_path = labels_dir,
    is_directory_recursive = False,
    file_type = "parquet",
    data_format = "3D",
    batch_size = batch_size,
    file_count = train_file_size,
    to_standardize= True,
    include_y_local= False,
    labels_list = ['x-midplane','y-midplane','cotAlpha','cotBeta'],
    scaling_list = [75.0, 18.75, 10.0, 1.22],
    input_shape = (2,13,21),
    transpose = (0,2,3,1),
    files_from_end=True,
    shuffle= True,

    load_from_tfrecords_dir = tfrecords_dir_train if load_from_tfrecords_enabled else None,
    tfrecords_dir = tfrecords_dir_train,
    use_time_stamps = (timeslices_val),
    max_workers = 1, # Don't make this too large (will use up all RAM)
    seed = 10,
    quantize = True # Quantization ON
)

validation_generator = OptimizedDataGenerator(
    data_directory_path = f"{data_base_dir}/dataset_2s_50x12P5_parquets/unflipped/recon3D/",
    labels_directory_path = f"{data_base_dir}/dataset_2s_50x12P5_parquets/unflipped/labels/",
    is_directory_recursive = False,
    file_type = "parquet",
    data_format = "3D",
    batch_size = val_batch_size,
    file_count = val_file_size,
    to_standardize= True,
    include_y_local= False,
    labels_list = ['x-midplane','y-midplane','cotAlpha','cotBeta'],
    scaling_list = [75.0, 18.75, 10.0, 1.22],
    input_shape = (2,13,21),
    transpose = (0,2,3,1),
    files_from_end=True,
    shuffle= True,

    load_from_tfrecords_dir = tfrecords_dir_val if load_from_tfrecords_enabled else None,
    tfrecords_dir = tfrecords_dir_val,
    use_time_stamps = (timeslices_val),
    max_workers = 1, # Don't make this too large (will use up all RAM)
    seed = 10,
    quantize = True # Quantization ON
)

print(f"input: {data_dir}")
print(f"input: {labels_dir}")
print(f"output: {tfrecords_dir_train}")
print(f"output: {tfrecords_dir_val}")

In [ ]:
# Create a numpy array that contains the recon3D and labels information
# X is the 20-timeslices or 2-timeslices recon3D data
# y is the labels data ['x-midplane','y-midplane','cotAlpha','cotBeta']
X_val_all = []
y_val_all = []

val_num_batches = validation_generator.__len__() # The total number of batches for the validation dataset
#val_num_batches = 1

for batch_num in range(val_num_batches): # Loop over all batches
    X_val, y_val = training_generator.__getitem__(batch_num) 
    X_val_all.append(X_val.numpy())
    y_val_all.append(y_val.numpy())

X_val_all = np.concatenate(X_val_all, axis=0)
y_val_all = np.concatenate(y_val_all, axis=0)

os.makedirs(npy_base_dir, exist_ok=True)
np.save(f"{npy_base_dir}/X_{timeslices_name}_val.npy", X_val_all)
np.save(f"{npy_base_dir}/y_{timeslices_name}_val.npy", y_val_all)

In [ ]:
# compiles model
model = CreateModel((13,21,2), n_filters=5, pool_size=3)
model.summary()

In [ ]:
model.compile(optimizer=Adam(learning_rate=0.001), loss=custom_loss)

In [ ]:
%%time
model_name = f"{dataset_name}_{sensor_geometry_name}_{timeslices_name}_{batch_size_name}" + ("_bnorm" if batch_norm_enabled else "")

best_model_hdf5 = f"{model_base_dir}/weights_7pitches/best_model_{model_name}.hdf5"
best_model_keras = f"{model_base_dir}/weights_7pitches/best_model_{model_name}.keras"
best_model_weights_hdf5 = f"{model_base_dir}/weights_7pitches/best_model_{model_name}_weights.hdf5"
best_model_weights_keras = f"{model_base_dir}/weights_7pitches/best_model_{model_name}_weights.keras"
best_model_architecture_json = f"{model_base_dir}/weights_7pitches/best_model_{model_name}_architecture.json"

if not load_model_from_hdf5_enabled:
    # training
    es = EarlyStopping(
        patience=50,
        restore_best_weights=True
    )

    checkpoint_base_dir = f"{model_base_dir}/weights_7pitches/{dataset_name}_{sensor_geometry_name}_{timeslices_name}_{batch_size_name}"  + ("_bnorm" if batch_norm_enabled else "") + "-checkpoints"

    os.makedirs(checkpoint_base_dir, exist_ok=True)
    checkpoint_filepath = checkpoint_base_dir + '/weights.{epoch:02d}-t{loss:.2f}-v{val_loss:.2f}.hdf5'
    mcp = tf.keras.callbacks.ModelCheckpoint(
        filepath=checkpoint_filepath,
        save_weights_only=True,
        monitor='val_loss',
        save_best_only=False,
    )

    class ScalePrintingCallback(keras.callbacks.Callback):    
        def on_epoch_end(self, epoch, logs=None):
            scale_layer = self.model.layers[-1]
            print(
                f"scaling layer ({epoch}):", 
                scale_layer.scale, 
                tf.math.softplus(scale_layer.scale)
            )

    print_scale = ScalePrintingCallback()

    history = model.fit(x=training_generator,
                        validation_data=validation_generator,
                        callbacks=[mcp],
                        epochs=100,
                        shuffle=False, # shuffling now occurs within the data-loader
                        verbose=1)

    # Revert to best model
    files = os.listdir(checkpoint_base_dir)
    vlosses = [float(f.split("-v")[1].split(".hdf5")[0]) for f in files]
    bestfile = files[np.argmin(vlosses)]
    model.load_weights(f"{checkpoint_base_dir}/{bestfile}")

    # Save (best) model information to file
    model.save(best_model_hdf5)
    model.save(best_model_keras)
    model.save_weights(best_model_weights_hdf5)
    model.save_weights(best_model_weights_keras)
    model_json = model.to_json()
    with open(best_model_architecture_json, "w") as json_file:
        json_file.write(model_json)

else:
    co = {"custom_loss": custom_loss}
    _add_supported_quantized_objects(co)
    # This overrides the previously compiled model
    # TODO: load just weights
    model = load_model(best_model_hdf5, custom_objects=co)
    model.summary()

In [ ]:
training_validation_loss_png = f"{model_base_dir}/weights_7pitches/training_validation_loss_{model_name}.png"
if load_model_from_hdf5_enabled:
    from PIL import Image
    img = Image.open(training_validation_loss_png)
    img.show()  # Opens the image in the default viewer
else: 
    plt.plot(history.history['loss'], label='Training Loss')
    plt.plot(history.history['val_loss'], label='Validation Loss')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.title('Training and Validation Loss')
    plt.legend()
    plt.savefig(training_validation_loss_png)  # Save as PNG
    plt.show()